<div class="alert alert-block alert-info" align="center">
    <h1>
        Imports
    </h1>
</div>

In [ ]:
import numpy as np
import mxnet as mx
from mxnet import nd, autograd, gluon
import pandas as pd
from keras.datasets import mnist
from keras.datasets import fashion_mnist
import time
from keras.datasets import cifar10

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.legend_handler import HandlerLine2D
from keras.utils import to_categorical

<div class="alert alert-block alert-info" align="center">
    <h1>
        Déclarations
    </h1>
</div>

<div align="center"><h1> Fonctions utiles </h1></div>

In [ ]:
def plot_histories (eta, epochs, cost_history, accuracy_history):
    fig, ax = plt.subplots(figsize = (5, 5))
    ax.set_ylabel(r'$J(\theta)$')
    ax.set_xlabel('Epochs')
    ax.set_title(r"$\eta$ :{}".format(eta))
    line1, = ax.plot(range(epochs), cost_history, label = 'Cost')
    line2, = ax.plot(range(epochs), accuracy_history, label = 'Accuracy')
    plt.legend(handler_map = {line1: HandlerLine2D(numpoints = 4)})

<div align="center"><h1> Classes </h1></div>

In [ ]:
class Conv3x3: # Une couche e convolution qui utilise un filtre de 3X3
    def __init__(self, num_filters):
        self.num_filters = num_filters

        # Les filtres dont des tableaux 3D de dimensions(num_filters, 3, 3)
        # On divise par 9 pour reduire la variance de notre valeur initiale
        self.filters = np.random.randn(num_filters, 3, 3) / 9

    def iterate_regions(self, image):
        '''
        Generates all possible 3x3 image regions using valid padding.
        - image is a 2d numpy array
        '''
        h, w = image.shape

        for i in range(h - 2):
            for j in range(w - 2):
                im_region = image[i:(i + 3), j:(j + 3)]
                yield im_region, i, j                

    def forward(self, input):
        '''
        Performs a forward pass of the conv layer using the given input.
        Returns a 3d numpy array with dimensions (h, w, num_filters).
        - input is a 2d numpy array
        '''
        if(input.ndim == 2): # 1er conv on recup l'image 2D
            self.last_input = input
            h, w = input.shape
            output = np.zeros((h - 2, w - 2, self.num_filters))
            for im_region, i, j in self.iterate_regions(input):
                output[i, j] = np.sum(im_region * self.filters, axis=(1, 2))

        else: # 2ème conv on récup le resultat du maxpool 3D
            self.last_input = input
            h, w , filters = input.shape
            output = np.zeros((h - 2, w - 2, self.num_filters))
            temp=np.transpose(input)
            for k in range(0, filters):
                for im_region, i, j in self.iterate_regions(temp[k]):
                    output[i, j] = np.sum(im_region * self.filters, axis=(1, 2))
                    
        return output
    
    def iterate_regions_back(self, image):
        '''
        Generates non-overlapping 2x2 image regions to pool over.
        - image is a 2d numpy array
        '''
        h, w, _ = image.shape
        new_h = h // 2
        new_w = w // 2

        for i in range(new_h):
            for j in range(new_w):
                im_region = image[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)]
                yield im_region, i, j

    def backprop(self, d_L_d_out, learn_rate):
        '''
        Performs a backward pass of the conv layer.
        - d_L_d_out is the loss gradient for this layer's outputs.
        - learn_rate is a float.
        '''
        if(self.last_input.ndim == 2): # 1er conv on recup l'image 2D
            d_L_d_filters = np.zeros(self.filters.shape)

            for im_region, i, j in self.iterate_regions(self.last_input):
                for f in range(self.num_filters):
                    d_L_d_filters[f] += d_L_d_out[i, j, f] * im_region

            # Update filters
            self.filters -= learn_rate * d_L_d_filters
            # We aren't returning anything here since we use Conv3x3 as
            # the first layer in our CNN. Otherwise, we'd need to return
            # the loss gradient for this layer's inputs, just like every
            # other layer in our CNN.
            return None

        else:
            d_L_d_input = np.zeros(self.last_input.shape)

            for im_region, i, j in self.iterate_regions_back(self.last_input):
                h, w, f = im_region.shape
                amax = np.amax(im_region, axis=(0, 1))

                for i2 in range(h):
                    for j2 in range(w):
                        for f2 in range(f):
                            # If this pixel was the max value, copy the gradient to it.
                            if im_region[i2, j2, f2] == amax[f2]:
                                d_L_d_input[i * 2 + i2, j * 2 + j2, f2] = d_L_d_out[i, j, f2]

            return d_L_d_input

        
class Relu:

    def __init__(self):
        self.params = []

    def forward(self, input):

        self.last_input = input
        output = np.maximum(0, input)  # element-wise
        
        #print('Forward relu')
        return output

    def backprop(self, dOut):
        """
        Backward propogation.
        f′(x) = {1 if x > 0}
                {0 otherwise}
        """
        dOut[dOut <= 0] = 0
        dOut[dOut > 0] = 1
        
        return dOut
    
class MaxPool2: # A Max Pooling layer using a pool size of 2.
    def iterate_regions(self, image):
        '''
        Generates non-overlapping 2x2 image regions to pool over.
        - image is a 2d numpy array
        '''
        h, w, _ = image.shape
        new_h = h // 2
        new_w = w // 2

        for i in range(new_h):
            for j in range(new_w):
                im_region = image[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)]
                yield im_region, i, j

    def forward(self, input):
        '''
        Performs a forward pass of the maxpool layer using the given input.
        Returns a 3d numpy array with dimensions (h / 2, w / 2, num_filters).
        - input is a 3d numpy array with dimensions (h, w, num_filters)
        '''
        self.last_input = input
        h, w, num_filters = input.shape
        output = np.zeros((h // 2, w // 2, num_filters))

        for im_region, i, j in self.iterate_regions(input):
            output[i, j] = np.amax(im_region, axis=(0, 1))

        return output

    def backprop(self, d_L_d_out):
        '''
        Performs a backward pass of the maxpool layer.
        Returns the loss gradient for this layer's inputs.
        - d_L_d_out is the loss gradient for this layer's outputs.
        '''
        d_L_d_input = np.zeros(self.last_input.shape)

        for im_region, i, j in self.iterate_regions(self.last_input):
            h, w, f = im_region.shape
            amax = np.amax(im_region, axis=(0, 1))

            for i2 in range(h):
                for j2 in range(w):
                    for f2 in range(f):
                        # If this pixel was the max value, copy the gradient to it.
                        if im_region[i2, j2, f2] == amax[f2]:
                            d_L_d_input[i * 2 + i2, j * 2 + j2, f2] = d_L_d_out[i, j, f2]

        return d_L_d_input

class Dropout:
    def __init__(self, drop_probability):
        self.drop_probability = drop_probability
    
    def forward(self,input):
        keep_probability = 1 - self.drop_probability
        mask = nd.random_uniform(0, 1.0, input.shape) < keep_probability
        mask=mask.asnumpy()
        #############################
        #  Avoid division by 0 when scaling
        #############################
        if keep_probability > 0.0:
            scale = (1/keep_probability)
        else:
            scale = 0.0
        out=input*mask*scale
        return out

class MyFlatten: # A Flattening layer
    def forward(self, input):
        self.last_input_shape = input.shape
        input = input.flatten()
        self.last_input = input
        return input
    
    def backprop(self, d_L_d_out):
        return d_L_d_out.reshape(self.last_input_shape)

class Dense: # A standard fully-connected layer with softmax activation.
    def __init__(self, input_len, nodes,**kwargs):
        self.activ_function_curr = kwargs.get("activation",None)
        self.activation_func=None
        # We divide by input_len to reduce the variance of our initial values
        self.weights = np.random.randn(input_len, nodes) / input_len
        self.biases = np.zeros(nodes)

    def forward(self, input):
        '''
        Performs a forward pass of the softmax layer using the given input.
        Returns a 1d numpy array containing the respective probability values.
        - input can be any array with any dimensions.
        '''
        self.last_input_shape = input.shape

        #input = input.flatten()
        self.last_input = input

        input_len, nodes = self.weights.shape

        totals = np.dot(input, self.weights) + self.biases
        self.last_totals = totals

        exp = np.exp(totals)
        return exp / np.sum(exp, axis=0)

    def backprop(self, d_L_d_out, learn_rate):
        '''
        Performs a backward pass of the softmax layer.
        Returns the loss gradient for this layer's inputs.
        - d_L_d_out is the loss gradient for this layer's outputs.
        - learn_rate is a float
        '''
        # We know only 1 element of d_L_d_out will be nonzero
        for i, gradient in enumerate(d_L_d_out):
            if gradient == 0:
                continue

            # e^totals
            t_exp = np.exp(self.last_totals)

            # Sum of all e^totals
            S = np.sum(t_exp)

            # Gradients of out[i] against totals
            d_out_d_t = -t_exp[i] * t_exp / (S ** 2)
            d_out_d_t[i] = t_exp[i] * (S - t_exp[i]) / (S ** 2)

            # Gradients of totals against weights/biases/input
            d_t_d_w = self.last_input
            d_t_d_b = 1
            d_t_d_inputs = self.weights

            # Gradients of loss against totals
            d_L_d_t = gradient * d_out_d_t

            # Gradients of loss against weights/biases/input
            d_L_d_w = d_t_d_w[np.newaxis].T @ d_L_d_t[np.newaxis]
            d_L_d_b = d_L_d_t * d_t_d_b
            d_L_d_inputs = d_t_d_inputs @ d_L_d_t

            # Update weights / biases
            self.weights -= learn_rate * d_L_d_w
            self.biases -= learn_rate * d_L_d_b
            return d_L_d_inputs


class MyCNN:
    def __init__(self):
        self.nbLayers = 0
        self.layers = [] # NN layers

    def printLayers(self):
        for i in range(len(self.CNN)):
            print(self.CNN[i].activationCNNFunc)
        for i in range(len(self.layers)):
            print(self.layers[i].activation_func)
        
    def info(self):
        print(f'Content of the network :');
        j = 0;
        for i in range(len(self.CNN)):
            print(f'\n\tLayer n° {i} du CNN => ')
            print(f'\t\tInput : {self.CNN[i].input}\n\t\tOutput : {self.CNN[i].output}')
            if (i != 0):
                print(f'\t\tCouche : {self.CNN[i].activationCNNFunc}')
                print(f'\t\tW shape : {self.CNN[i].parameters["W"].shape}\n')
                print(f'\t\tb shape : {self.CNN[i].parameters["b"].shape}\n')
                
        for i in range(len(self.layers)):
            print(f'\n\tLayer n° {i} du NN => ')
            print(f'\t\tInput : {self.layers[i].input}\n\t\tOutput : {self.layers[i].output}')
            if (i != 0):
                print(f'\t\tCouche : {self.layers[i].activation_func}')
                print(f'\t\tW shape : {self.layers[i].parameters["W"].shape}\n')
                print(f'\t\tb shape : {self.layers[i].parameters["b"].shape}\n')
        
    def addLayer(self, layer):
        self.nbLayers += 1 # Le layer 0 = input, si CNN = images
        self.layers.append(layer)
        
     
    def forward_propagation(self, X):
        outPrevious=0
        for num_layer in range (0, self.nbLayers):
            if (type(self.layers[num_layer]) is Conv3x3):
                if num_layer==0:
                    X=(X / 255) - 0.5
                    outPrevious=self.layers[num_layer].forward(X)
                else : 
                    outPrevious=self.layers[num_layer].forward(outPrevious)
            
            if (type(self.layers[num_layer]) is Relu):
                outPrevious=self.layers[num_layer].forward(outPrevious)
                      
            if (type(self.layers[num_layer]) is MaxPool2):
                outPrevious=self.layers[num_layer].forward(outPrevious) 
              
            if (type(self.layers[num_layer]) is Dropout):
                outPrevious=self.layers[num_layer].forward(outPrevious)
                  
            if (type(self.layers[num_layer]) is MyFlatten):
                outPrevious=self.layers[num_layer].forward(outPrevious) 
                
            if (type(self.layers[num_layer]) is Dense):
                
                outPrevious=self.layers[num_layer].forward(outPrevious)
        return outPrevious
               
        
        
    def cost_function(self, out,y):
        return (-np.log(out[y]))

    def backward_propagation(self, out,y,eta):
        previousGradient=0
        for num_layer in range (self.nbLayers-1,0,-1):
            if (type(self.layers[num_layer]) is Dense):
                # initialisation du gradient à la fin
                gradient = np.zeros(10)
                gradient[y] = -1 / out[y]
                previousGradient = self.layers[num_layer].backprop(gradient, eta)
                #print ("backpropagation - softmax", previousGradient.shape)
            
            if (type(self.layers[num_layer]) is MyFlatten):
                previousGradient = self.layers[num_layer].backprop(previousGradient)
                #print ("backpropagation - flat", previousGradient.shape) 
                      
            if (type(self.layers[num_layer]) is MaxPool2):
                previousGradient = self.layers[num_layer].backprop(previousGradient)
                #print ("backpropagation - pool", previousGradient.shape) 
            if (type(self.layers[num_layer]) is Relu):
                previousGradient = self.layers[num_layer].backprop(previousGradient)
                #print ("backpropagation - relu", previousGradient.shape) 
            if (type(self.layers[num_layer]) is Conv3x3):
                #print ("backpropagation - Conv 3x3")        
                previousGradient = self.layers[num_layer].backprop(previousGradient, eta)
 
    def convert_prob_into_class(self,probs):
        probs = np.copy(probs)#pour ne pas perdre probs, i.e. y_hat
        probs[probs > 0.5] = 1
        probs[probs <= 0.5] = 0
        return probs



    def accuracy(self,out, y):
        acc = 1 if np.argmax(out) == y else 0
        return acc       

    def predict(self, X):
        outPrevious=self.forward_propagation(X)
        return outPrevious

    

    def fit(self, X, y, *args,**kwargs):    
        epochs=kwargs.get("epochs",20)
        verbose=kwargs.get("verbose",False)
        eta =kwargs.get("eta",0.01)
        cost_history = []
        accuracy_history = []
        timo = []
        t = np.zeros(epochs)
        for nb_epochs in range(epochs):
            t[nb_epochs]=time.time()
            if nb_epochs > 0:
                duration = t[nb_epochs]- t[nb_epochs - 1]
            else:
                duration = 0
            timo.append(duration)
            cost_all_images=[]
            acc_all_images=[]
            cost = 0
            acc = 0
            print ("run epoch: ",nb_epochs+1)
            for i, (im, label) in enumerate(zip(X, y)):
                # Do a forward pass.
                startTime = time.time()
                out=self.forward_propagation(im)
                cost+=self.cost_function(out,label)
                cost_all_images.append(self.cost_function(out,label))
                acc+=self.accuracy(out, label)
                acc_all_images.append(self.accuracy(out, label))
                self.backward_propagation(out,label,eta)
                endTime = time.time()
                if verbose:
                    if i % 100 == 99:
                      print('[Step %d] Past 100 steps: Average Loss %.3f | Accuracy: %d%%  ' %(i + 1, cost / 100, acc))
                      #print(f'Step \t{i + 1} | Past 100 steps: Average Loss :{cost / 100:.2f} |Accuracy: {acc} %| Time : {endTime - startTime:.2f}\n')
                      cost = 0
                      acc = 0
            current_cost=np.average(cost_all_images)
            cost_history.append(current_cost)  
            current_acc=np.average(acc_all_images)
            accuracy_history.append(current_acc)
            #timo.append(duration)
        
        return cost_history, accuracy_history
 

<div class="alert alert-block alert-info" align="center">
    <h1>
        Applications
    </h1>
</div>

<div align="center"><h1> Classification des données de mnist </h1></div>

In [ ]:
epochs = 8
eta = 0.01
num_classes = 10
# the data, split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# prendre moins d'images
X_train=X_train[0:2000]
y_train=y_train[0:2000]
X_test=X_test[0:800]
y_test=y_test[0:800]

print ("Learning on:", X_train.shape[0], " examples")
print ("Test on:",X_test.shape[0]," examples")

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
#CONSTRUCTION DU RESEAU
network = MyCNN()
network.addLayer(Conv3x3(8))
network.addLayer(Relu())
network.addLayer(MaxPool2()) 
network.addLayer(Dropout(0.1))
network.addLayer(MyFlatten())
network.addLayer(Dense(13 * 13 * 8, num_classes,activation="softmax"))

cost_history,accuracy_history=network.fit(X_train, y_train, verbose=True, epochs=epochs)

accuracy_test=[]
timo = []
t = np.zeros(len(X_test))

for i in range (len(X_test)):
    t[i] = time.time()
    y_pred=network.predict(X_test[i])
    acc_test = network.accuracy(y_pred, y_test[i])
    accuracy_test.append(acc_test)

    
print("Accuracy test: %.3f"%np.average(accuracy_test))
print("time test:",t[len(X_test)-1]-t[0])
# Affichage des historiques
plot_histories (eta,epochs,cost_history,accuracy_history)



<div align="center"><h1> Classification des données de fashion mnist </h1></div>

In [ ]:
epochs = 8
eta = 0.01
((X_train, y_train), (X_test, y_test)) = fashion_mnist.load_data()

# prendre moins d'images
X_train=X_train[0:2000]
y_train=y_train[0:2000]
X_test=X_test[0:800]
y_test=y_test[0:800]

print ("Learning on:", X_train.shape[0], " examples")
print ("Test on:",X_test.shape[0]," examples")

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
network = MyCNN()
network.addLayer(Conv3x3(8))
network.addLayer(Relu())
network.addLayer(MaxPool2())
network.addLayer(Dropout(0.1))
network.addLayer(MyFlatten())
network.addLayer(Dense(13 * 13 * 8, 10,activation="softmax"))
cost_history,accuracy_history=network.fit(X_train, y_train, verbose=True, epochs=epochs)

t = np.zeros(len(X_test))
accuracy_test=[]
for i in range (len(X_test)):
    t[i] = time.time()
    y_pred=network.predict(X_test[i])
    acc_test = network.accuracy(y_pred, y_test[i])
    accuracy_test.append(acc_test)
    
print("Accuracy test: %.3f"%np.average(accuracy_test))
print("time test:",t[len(X_test)-1]-t[0])

# Affichage des historiques
plot_histories (eta,epochs,cost_history,accuracy_history)

<div align="center"><h1> Classification des données de Cifar 10 </h1></div>

In [ ]:
epochs = 8
eta = 0.01
num_classes = 10
# The data, split between train and test sets:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# prendre moins d'images
X_train=X_train[0:2000]
y_train=y_train[0:2000]
X_test=X_test[0:800]
y_test=y_test[0:800]
print ("Learning on:", x_train.shape[0], " examples")
print ("Test on:",x_test.shape[0]," examples")
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
network = MyCNN()
network.addLayer(Conv3x3(16))# (32,32,16)
network.addLayer(Relu())     # (32,32,16)
network.addLayer(MaxPool2())  # (16,16,8)
network.addLayer(Dropout(0.1)) # (16,16,8)
network.addLayer(MyFlatten())  # (2048)
network.addLayer(Dense(3600, 10,activation="softmax"))

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255


cost_history,accuracy_history=network.fit(X_train, y_train, verbose=True, epochs=epochs)

accuracy_test=[]
t = np.zeros(len(X_test))
for i in range (len(X_test)):
    t[i] = time.time()

    y_pred=network.predict(X_test[i])
    acc_test = network.accuracy(y_pred, y_test[i])
    accuracy_test.append(acc_test)
    
print("Accuracy test: %.3f"%np.average(accuracy_test))
print("time test:",t[len(X_test)-1]-t[0])
# Affichage des historiques
plot_histories (eta,epochs,cost_history,accuracy_history)


 <div class="alert alert-block alert-info" align="center">
    <h1>
        Data Generator a REVOIR SI ON N'A LE TEMPS

   </h1>
</div>


In [ ]:
epochs = 8
eta = 0.01
num_classes = 10
# the data, split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# prendre moins d'images
X_train=X_train[0:2000]
y_train=y_train[0:2000]
X_test=X_test[0:800]
y_test=y_test[0:800]
print ("Learning on:", X_train.shape[0], " examples")
print ("Test on:",X_test.shape[0]," examples")
print ("Learning on:", X_train.shape[0], " examples")
print ("Test on:",X_test.shape[0]," examples")
network = MyCNN()
network.addLayer(Conv3x3(8))
network.addLayer(Relu())
network.addLayer(MaxPool2())
network.addLayer(Dropout(0.1))
network.addLayer(MyFlatten())
network.addLayer(Dense(13 * 13 * 8, num_classes,activation="softmax"))
#print("X_train avant datagen",X_train)
#cost_history,accuracy_history=network.fit(X_train, y_train, verbose=True, epochs=epochs)

batch_size = 32

# Convert class vectors to binary class matrices.
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)


# initiate RMSprop optimizer
#opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Let's train the model using RMSprop
#model.compile(loss='categorical_crossentropy',
#              optimizer=opt,
#              metrics=['accuracy'])

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape)
img_rows, img_cols = 28, 28
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols,1)
print(X_train.shape)

print(X_train.shape)
print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)


# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(X_train)
#print("X_train apres datagen",X_train)

#network.fit_generator(datagen.flow(X_train, y_train, batch_size=32),steps_per_epoch=len(x_train) / 32, epochs=epochs)
#print("X_train apres datagen",X_train)
# Fit the model on the batches generated by datagen.flow().
#cost_history,accuracy_history=network.fit(X_train, y_train,validation_data=(X_test, y_test),epochs=epochs,workers=4)
#cost_history,accuracy_history=network.fit(X_train, y_train, verbose=True, epochs=epochs)



'''

accuracy_test=[]
timo = []
t = np.zeros(len(X_test))

for i in range (len(X_test)):
    t[i] = time.time()

        
    y_pred=network.predict(X_test[i])
    acc_test = network.accuracy(y_pred, y_test[i])
    accuracy_test.append(acc_test)
    #timo.append(duration)


print("Accuracy test: %.3f"%np.average(accuracy_test))
#print("time test: %.3f"%np.average(timo))
print("time test:",t[len(X_test)-1]-t[0])
# Affichage des historiques
plot_histories (eta,epochs,cost_history,accuracy_history)

'''